<a href="https://colab.research.google.com/github/caiodavic/Processamento-de-Linguagem-Natural-2022.1/blob/main/Classifica%C3%A7%C3%A3o_de_Texto_com_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Sátiras com Word Embedding
- Vamos classificar um dataset que contém notícias reais e sátiras utilizando Word Embeddings. Após a classificação, vamos extrair os pesos da primeira camada (Camada de Embedding) da rede neural e criar nossa matriz de Embeddings. 

### Importando as Bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# com essa célula o google colab consegue ler arquivos direto do google drive. 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Leitura de Dados e Divisão dos Dados

In [ ]:
# é importante alterar o caminho e colocar para onde o arquivo está salvo no seu drive
data = pd.read_csv('/content/gdrive/MyDrive/2022.1/PLN/csv_satiras_reais.csv')
data.head()

,title,text,label
0,crise e tao grande que nem tiozao do pave fez...,a familia guimaraes passou a noite de natal pe...,satire
1,nao me representam diz jesus sobre intolerant...,uma menina de 11 anos apedrejada ao sair de um...,satire
2,marina silva e heloisa helena montam novo par...,insatisfeitas com seus partidos com as siglas ...,satire
3,dez propostas que podem realmente mudar o brasil,o instituto nupal nucleo de pesquisas da ameri...,satire
4,apresentadora do cidade alerta bahia dara cur...,assassinatos sequestros mortes violentas. nen...,satire


In [ ]:
x = data['text']
y = [1 if i == 'satire' else 0 for i in data['label']]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

### Preparação dos dados

#### Tokenização, criação do vocabulário e "padded"
- A fim de transformar nossos dados  para que a primeira camada possa receber, precisamos tokenizar, e fazer com que todos os vetores de sentenças tenham a mesma dimensão.


In [ ]:
from keras.preprocessing.text import Tokenizer
vocab_size = 10000
oov_token = "<OOV>"
embedding_dim = 64
max_length = 120
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)

- Vamos criar nosso tokenizador com os dados da coluna de texto do treinamento 

In [ ]:
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [ ]:
len(word_index)

62125

- Agora precisamos mapear nossas palavras para o índice que o tokenizador criou.

In [ ]:
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

- Com as palavras mapeadas para valores, precisamos agora tornar todas as sequências do mesmo tamanho. Temos um tamanho máximo de 100. Vamos utilizar `padding`e `truncating`com o valor 'post', isso significa que tanto para truncar a sentença ou aumentar o seu vetor será feito no final do vetor.

In [ ]:
x_train_padded = pad_sequences(x_train_sequences, maxlen = max_length, padding = 'post',truncating='post')
x_test_padded = pad_sequences(x_test_sequences, maxlen= max_length, padding = 'post',truncating='post')

### Construindo a Rede Neural
- Vamos treinar a rede neural para classificar nossos textos. Uma vez que a rede neural está treinada, podemos extrair os pesos da primeira camada, assim teremos nossos embeddings.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64,activation='relu')))
model.add(tf.keras.layers.Dense(6,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 64)           640000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              16512     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 6)                 774       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 657,293
Trainable params: 657,293
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train_padded,np.array(y_train),epochs=10,validation_data=(x_test_padded,np.array(y_test)))

Epoch 1/10
250/250 [==============================] - 18s 64ms/step - loss: 0.4980 - accuracy: 0.7609 - val_loss: 0.3154 - val_accuracy: 0.8840
Epoch 2/10
250/250 [==============================] - 15s 59ms/step - loss: 0.2600 - accuracy: 0.9087 - val_loss: 0.3159 - val_accuracy: 0.8615
Epoch 3/10
250/250 [==============================] - 15s 60ms/step - loss: 0.1185 - accuracy: 0.9592 - val_loss: 0.1490 - val_accuracy: 0.9475
Epoch 4/10
250/250 [==============================] - 17s 66ms/step - loss: 0.0354 - accuracy: 0.9891 - val_loss: 0.1545 - val_accuracy: 0.9555
Epoch 5/10
250/250 [==============================] - 15s 62ms/step - loss: 0.0139 - accuracy: 0.9960 - val_loss: 0.1783 - val_accuracy: 0.9500
Epoch 6/10
250/250 [==============================] - 17s 68ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.1595 - val_accuracy: 0.9595
Epoch 7/10
250/250 [==============================] - 16s 64ms/step - loss: 0.0012 - accuracy: 0.9999 - val_loss: 0.1795 - val_accuracy:

In [ ]:
loss, accuracy = model.evaluate(x_test_padded, np.array(y_test))
print(f'A acurácia foi de: {accuracy}')

63/63 [==============================] - 1s 14ms/step - loss: 0.2311 - accuracy: 0.9540
A acurácia foi de: 0.9539999961853027


### Extração dos pesos
- Extraindo os pesos, podemos exportar o vetor de cada palavra do nosso vocabulário

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 64)


In [ ]:
reversed_word_index = {}
for i in word_index.keys():
  reversed_word_index[word_index[i]] = i

reversed_word_index

In [ ]:
import io

out_vectors = io.open('vecs.txt', 'w', encoding='utf-8')
out_vectors.write(f'{vocab_size} {embedding_dim}\n')

for word_num in range(1,vocab_size+1):
  
  word = reversed_word_index[word_num]
  embeddings = weights[word_num-1]
  line_embedding =  word +' ' + ' '.join([str(x) for x in embeddings])+ '\n'
  if word_num == vocab_size:
    line_embedding = line_embedding.rstrip()  
  out_vectors.write(line_embedding)
  
out_vectors.close()

In [ ]:
from gensim.models import KeyedVectors
model_embedding = KeyedVectors.load_word2vec_format('vecs.txt')

In [ ]:
model_embedding.get_vector('amor')

array([-7.7490255e-02,  2.8578673e-02, -2.9935014e-02,  4.0734537e-02,
        3.7374351e-02,  4.3725137e-02, -4.3079942e-02,  2.5085259e-02,
       -3.6693448e-03, -7.5272828e-02, -6.2611312e-02,  3.5357263e-02,
        4.4600915e-02,  4.1194756e-02,  1.0161753e-01,  7.6673083e-02,
        3.5793759e-02,  2.5541573e-05, -3.2469507e-02,  2.0374404e-02,
        2.2557979e-02, -7.0923664e-02, -5.7299502e-02,  6.3124470e-02,
        1.1048226e-02,  1.0526380e-02,  2.8292488e-02,  2.0341251e-02,
        1.6981418e-03, -3.1631985e-03,  8.5497379e-02,  6.4519249e-02,
       -3.5611324e-03,  1.4506328e-02,  5.6007400e-02,  4.0687256e-02,
       -4.0227380e-02,  8.5500693e-03,  1.8519528e-02, -5.9586897e-02,
       -5.5159714e-02,  1.1555127e-02,  1.1547644e-02,  4.0947422e-02,
       -5.1473878e-02, -5.0669424e-02, -6.3177362e-02,  5.4570053e-02,
       -9.0539356e-04, -4.7544021e-02,  6.8863913e-02,  9.4348982e-02,
        5.3891331e-02,  3.5973992e-02, -3.8921371e-02,  4.7982961e-02,
      

In [ ]:
model_embedding.most_similar(negative=['amor'])

[('militancia', 0.5878221988677979),
 ('verao', 0.5773073434829712),
 ('grandes', 0.5753006935119629),
 ('saida', 0.5650980472564697),
 ('excecao', 0.5454781651496887),
 ('juntou', 0.5452638864517212),
 ('17', 0.5323375463485718),
 ('atrair', 0.5282691717147827),
 ('restricao', 0.5244694948196411),
 ('apresentador', 0.5198777914047241)]

In [ ]:
with open('tensors.tsv', 'w') as tensors:
    with open('metadata.tsv', 'w') as metadata:
        for word in model_embedding.wv.index2word:
            metadata.write(word + '\n')
            vector_row = '\t'.join(map(str, model_embedding.wv[word]))
            tensors.write(vector_row + '\n')

<ipython-input-36-5091996d392d>:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  for word in model_embedding.wv.index2word:
<ipython-input-36-5091996d392d>:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector_row = '\t'.join(map(str, model_embedding.wv[word]))


In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('tensors.tsv')
  files.download('metadata.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>